<a href="https://colab.research.google.com/github/AdmiralHonda/ml_intro/blob/main/ch04/emotion_analytics_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 下準備（ubuntu環境なのでaptでインストールしてます）
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null # mecabの利用に必要なライブラリのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null                    # gitから辞書ファイルのクローン
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1                   # クローンした辞書のインストール
!pip install mecab-python3==0.7 > /dev/null                                                             # 0.7以外だと謎のエラーが発生して安定しない(RunTime Error)

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc                                                              # 辞書の参照先にインストール先のディレクトリを追加
!echo `mecab-config --dicdir` "/mecab-ipadic-neologd"                                                   # mecabの設定ファイルに新しく辞書を追加したことを追記

# 感情分析でネットのコメントに対して分析をしてみよう

今回の演習ではとあるニュースに対してのコメントから最も批判的、また好意的なコメントを抽出するということをやってもらいます。  
SNSなどで炎上したときにコメントなどを追い、どんな反応が出ているのかを数値化して定量的に量ることで、他のトピックと比べてどれくらい世間が反応しているかや、有益なコメントを抽出することもできます。  
今回取り組んでもらう感情分析のやり方としては、分析対象の文章中に対して否定的、肯定的な単語が何個あったかを数えて評価する方式を採用します。ほかにも機械学習を使用するやり方があります。  
そのために後述する「日本語評価極性辞書」という日本語の単語に対して肯定的か否定的かなどの評価付けをしてある辞書を使用し、分析対象の単語に対してこの辞書から検索して評価を行っていくことになります。  

<br>

今回の手順:

1. データの取得　　　　　　　　　　　　　（実行するだけ）
2. 感情分析辞書の作成　　　　　　　　　　（実行するだけ）
3. データの前処理　　　　　　　　　　　　（演習）
4. データを分析にかけ、特徴ベクトルに変換（演習）
5. 特徴ベクトルをもとにコメントを抽出　　（演習）  

今回の抽出は[この記事](https://resanaplaza.com/2022/06/04/__trashed-2/)を参考にしています。ただ、どういった特徴ベクトルが目的のコメントに近いかは自分たちで判断し、考察してもらいます。  


In [ ]:
# @title データの取得

import json
import urllib.request

# ネット上にホストしてある俺のサーバからjson形式のデータを取得
with urllib.request.urlopen('https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/emotion_practice/emotion_practice.json') as response:
   comments = json.load(response)

# サンプルを提示
print(comments[0:1]) 

In [ ]:
# @title 感情分析辞書の作成

with urllib.request.urlopen('https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/emotion_practice/pn.csv.m3.120408.trim') as response:
   lines = response.read().decode("utf-8").split("\n")

emotion_dic = {}
for x in lines:
  word_dic = x.split("\t")
  try:
    emotion_dic[word_dic[0]] = word_dic[1]
  except IndexError:
    print(word_dic)


In [ ]:
# @title データの前処理
"""
commentsの中身は改行コードなどの特殊文字に対しての処理をしてないです
\t,\r,\nなどを削除しましょう
"""

# データを特徴ベクトルに変換

参考サイトにあるような特徴ベクトルを作成し、分析対象のコメントを定量的に量れる、また計算可能な形式へ変換します。   

> **特徴ベクトルとは？**  
ベクトルの要素一つ一つに重さや大きさ、年齢や点数などの対象する概念を特徴づける数値を割り当てたベクトル  
例）車 -> (排気量,販売年数,燃費) -> (660,2011,15.4)

今回使用するまた基礎とする特徴ベクトルとは以下のようなものです。  

(肯定的な単語数,否定的な単語数,中性的な単語数,判定不能な単語数)   -> 例)(34,5,65,23)

以上の4つの要素を持つベクトルとなります。

## 日本語評価極性辞書

今回感情分析に使用する辞書について簡単に説明します。  
辞書の元データは項目をタブで区切り、データ自体を開業で区切るtsv形式のファイルになっています。以下の形式でデータが格納されています。  
単語\t評価\t状態\n  
評価の部分に否定的か肯定的なのかの評価がラベル付けされる形で格納されています。最後の状態は行為を表すものか感覚として感じるものであるかなどの補足情報が付与されています。  

評価に関しては以下の種類のラベルがあります。  
- p
肯定的な単語
- n
否定的な単語
- e
中性的な単語

今回はこの辞書を単語をキーとして値が評価ラベルを返す辞書型の変数`emotion_dic`として使用します。  

In [ ]:
# @title データを特徴ベクトルに変換

In [ ]:
# @title 特徴ベクトルをもとに最も肯定的、否定的なコメントを抽出